# Assignment 2: IR

## Preparations
* Put all your imports, and path constants in the next cells
* Make sure ***MATERIALS_DIR*** points to the directory where you extracted the Zip file.
* Make sure all your paths are **relative to ** ***MATERIALS_DIR*** and **NOT hard-coded** in your code.

In [ ]:
# imports
# Put all your imports here

from whoosh import index, writing, qparser
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import *
#from whoosh.qparser import QueryParser
import os, os.path
import shutil
# NLTK Library
import nltk
from nltk.stem import *
nltk.download("wordnet")

In [ ]:
MATERIALS_DIR = r"C:\DSS_Fall2017_Assign2"
#
# Put other path constants here
#
DOCUMENTS_DIR = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\documents")
INDEX_DIR = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\index1")
QUER_FILE = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\topics\gov.topics")
QRELS_FILE = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\qrels\gov.qrels")
OUTPUT_FILE = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\myres")
TREC_EVAL = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\trec_eval\trec_eval.exe")
INDEX_DIR2 = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\index2")
OUTPUT_FILE2 = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\myres2")


In [ ]:
class CustomFilter(Filter):
    is_morph = True
    def __init__(self, filterFunc, *args, **kwargs):
        self.customFilter = filterFunc
        self.args = args
        self.kwargs = kwargs
    def __eq__(self):
        return (other
                and self.__class__ is other.__class__)
    def __call__(self, tokens):
        for t in tokens:
            if t.mode == 'query': # if called by query parser
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t
            else: # == 'index' if called by indexer
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t

### Code for trec_eval measures - TO BE DELETED
!$TREC_EVAL -h -m official


## Question 1
Provide your text answers in the following two markdown cells

### Q1 (a): Provide answer to Q1 (a) here [markdown cell]

recip_rank



### Q1 (b): Provide answer to Q1 (b) here [markdown cell]
Government websites are usually customer focused, where the person is only interested in finding the information they are searching for. Usually this information can be retrievied from one relevant document.

Thus the higher the relevant document ranks, the more satisfied the customer is making the search engine better

## Question 2

### Q2 (a): Write your code below

In [ ]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q2, your query parser in QP_Q2, and your searcher in SEARCHER_Q2

mySchema = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = RegexTokenizer()))

if os.path.isdir(INDEX_DIR):
    shutil.rmtree(INDEX_DIR)

# create the directory for the index
os.makedirs(INDEX_DIR)

# create index
myIndex = index.create_in(INDEX_DIR, mySchema)

filesToIndex = []
for root, dirs, files in os.walk(DOCUMENTS_DIR):
    filePaths = [os.path.join(root, fileName) for fileName in files if not fileName.startswith('.')]
    filesToIndex.extend(filePaths)


In [ ]:

myWriter = writing.BufferedWriter(myIndex, period=60, limit=1000)
try:
    # write each file to index
    for docNum, filePath in enumerate(filesToIndex):
        with open(filePath, "r", encoding="utf8") as f:
            fileContent = f.read()
            myWriter.add_document(file_path = filePath,
                                  file_content = fileContent)
            
            if (docNum % 1000 == 0):
                print("already indexed:", docNum+1)
    print("done indexing.")

finally:
    # save the index
    myWriter.close()

In [ ]:

myQueryParser = QueryParser("file_content", schema=myIndex.schema)
mySearcher = myIndex.searcher()

In [ ]:
# Load topic file - a list of topics(search phrases) used for evalutation
topicsFile = open(QUER_FILE,"r")
topics = topicsFile.read().splitlines()
print(topics)

# create an output file to which we'll write our results
outputTRECFile = open(OUTPUT_FILE, "w")

# for each evaluated topic:
# build a query and record the results in the file in TREC_EVAL format
for topic in topics:
    topic_id, topic_phrase = tuple(topic.split(" ", 1))
    topicQuery = myQueryParser.parse(topic_phrase)
    topicResults = mySearcher.search(topicQuery, limit=None)
    for (docnum, result) in enumerate(topicResults):
        score = topicResults.score(docnum)
        outputTRECFile.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))

# close the topic and results file
outputTRECFile.close()
topicsFile.close()

In [ ]:
INDEX_Q2 = myIndex
QP_Q2 = myQueryParser
SEARCHER_Q2 = mySearcher

#INDEX_Q2 = None # Replace None with your index for Q2
#QP_Q2 = None # Replace None with your query parser for Q2
#SEARCHER_Q2 = None # Replace None with your searcher for Q2

In [ ]:
!cat $QRELS_FILE
### Relevant or Not - human judgement

In [ ]:
!cat $OUTPUT_FILE
### My results

In [ ]:
!$TREC_EVAL -q $QRELS_FILE $OUTPUT_FILE
###Comparison of the two

### Q2 (b): Provide answer to Q2 (b) here [markdown cell]

For recip_rank the average score was 0.2196

### Q2 (c): Provide answer to Q2(c) here [markdown cell]
Topics it did very well:
18
24
Topics it did poorly on:
2,28,6,79

Is a bad case where the score is 0 or >0 but low?



## Question 3

### Q3 (a): Provide answer to Q3 (a) here [markdown cell]
Chosen Topic: Topic #4 - "*to be filled in*"

For this topic there were a total of 4 relevant documents but the original query retreived 12 documents. Of those retreived there was 1 True Positive case and 11 False Positive cases. As there were 4 relevant documents of which only 1 was True Positive, the rest of the 3 documents are classified as False Negative Cases.

An example of each is given below:

False Positive Case: In the Output_File produced above, Document G00-85-1525415 received a score of ~13.4 and ranked 2nd after using Whoosh's baseline system. However the Qrels file gives a relevance of 0 for topic #4. Thus the system assumed the document to be relevant even though it shouldn't have.

False Negative Case: In the Qrels file, document G00-36-1275993 is considered relevant for topic #4 based off human judgement. However in the OutputFile produced above this document does not receive any score or rank for this topic. This means the Whoosh system considered this document to be irrelevant for topic #4 which isn't actually the case.

Using Text Analyzers such as a stem, stop or lowercase filter prior to using the whoosh system would help improve performance. The data would be cleaned of 'filler' words, ignore cases and only partial words to increase the likelihood of matching documents and finding relevant ones.


### Q3 (b): Write your code below

In [ ]:
###GET RID OF THIS CELL

import re
myAnalyzer = RegexTokenizer() | LowercaseFilter() | IntraWordFilter() | StopFilter() |  CustomFilter(SnowballStemmer("english").stem)
topicsFile = open(QUER_FILE,"r")
topics = topicsFile.read().splitlines()
topics2 = []
for topic1 in topics:
    topic1 = re.sub(r'\d+', '', topic1)
    topics2.append(topic1)
    print(topic1)
    print([token.text for token in myAnalyzer(topic1)])
    print("\n")
print(topics2)

In [ ]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q3, your query parser in QP_Q3, and your searcher in SEARCHER_Q3

myAnalyzer = RegexTokenizer() | LowercaseFilter() | IntraWordFilter() | StopFilter() | CustomFilter(SnowballStemmer("english").stem)

mySchema2 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = myAnalyzer))

if os.path.isdir(INDEX_DIR2):
    shutil.rmtree(INDEX_DIR2)


# create the directory for the index
os.makedirs(INDEX_DIR2)

# create index
myIndex2 = index.create_in(INDEX_DIR2, mySchema2)

filesToIndex = []
for root, dirs, files in os.walk(DOCUMENTS_DIR):
    filePaths = [os.path.join(root, fileName) for fileName in files if not fileName.startswith('.')]
    filesToIndex.extend(filePaths)





In [ ]:

myWriter2 = writing.BufferedWriter(myIndex2, period=60, limit=1000)
try:
    # write each file to index
    for docNum, filePath in enumerate(filesToIndex):
        with open(filePath, "r", encoding="utf8") as f:
            fileContent = f.read()
            myWriter2.add_document(file_path = filePath,
                                  file_content = fileContent)
            
            if (docNum % 1000 == 0):
                print("already indexed:", docNum+1)
    print("done indexing.")

finally:
    # save the index
    myWriter2.close()

In [ ]:
myQueryParser2 = qparser.QueryParser("file_content", schema=myIndex2.schema,group=qparser.OrGroup)
mySearcher2 = myIndex2.searcher()

In [ ]:
# Load topic file - a list of topics(search phrases) used for evalutation
topicsFile = open(QUER_FILE,"r")
topics = topicsFile.read().splitlines()
    
# create an output file to which we'll write our results
outputTRECFile2 = open(OUTPUT_FILE2, "w")

# for each evaluated topic:
# build a query and record the results in the file in TREC_EVAL format
for topic in topics:
    topic_id, topic_phrase = tuple(topic.split(" ", 1))
  #  print(topic_phrase)
    topicQuery = myQueryParser2.parse(topic_phrase)
   # print(topicQuery)
    topicResults = mySearcher2.search(topicQuery, limit=None)
    for (docnum, result) in enumerate(topicResults):
        score = topicResults.score(docnum)
        outputTRECFile2.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))

# close the topic and results file
outputTRECFile2.close()
topicsFile.close()

In [ ]:
INDEX_Q3 = myIndex2 # Replace None with your index for Q3
QP_Q3 = myQueryParser2 # Replace None with your query parser for Q3
SEARCHER_Q3 = mySearcher2 # Replace None with your searcher for Q3

In [ ]:
!cat $OUTPUT_FILE2

In [ ]:
!$TREC_EVAL -q $QRELS_FILE $OUTPUT_FILE2

### Q3 (c): Provide answer to Q3 (c) here [markdown cell]

I used an additional 5 Text Analyzers:
1) LowercaseFilter() - Used this to make all the words in the query lowercase
2) IntraWordFilter() - Used this to remove any punctuation within the query
3) StopFilter() - Used this to removes any filler words such as 'and' 'are' etc. that were present in the queries 
4) CustomFilter(SnowballStemmer("english").stem)

### Q3 (d): Provide answer to Q3 (d) here [markdown cell]

### Q3 (e): Provide answer to Q3 (e) here [markdown cell]

### Q3 (f): Provide answer to Q3 (f) here [markdown cell]

## Question 4 (Graduate Students)

In [ ]:
GRAD_STUDENT = False # change to True if you are a grad student

### Q4 (a): Provide answer to Q4 (a) here [markdown cell]

### Q4 (b): Write your code below

In [ ]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q4, your query parser in QP_Q4, and your searcher in SEARCHER_Q4

In [ ]:
INDEX_Q4 = None # Replace None with your index for Q4
QP_Q4 = None # Replace None with your query parser for Q4
SEARCHER_Q4 = None # Replace None with your searcher for Q4

### Q4 (c): Provide answer to Q4 (a) here [markdown cell]

### Q4 (d): Provide answer to Q4 (a) here [markdown cell]

### Q4 (e): Provide answer to Q4 (a) here [markdown cell]

### Q4 (f): Provide answer to Q4 (a) here [markdown cell]

## Validation

In [ ]:
# Run the following cells to make sure your code returns the correct value types

In [ ]:
from whoosh.index import FileIndex
from whoosh.qparser import QueryParser
from whoosh.searching import Searcher
import os.path

### Path Validation

In [ ]:
assert "MATERIALS_DIR" in globals(), "variable MATERIALS_DIR does not exists"
assert(os.path.isdir(os.path.join(MATERIALS_DIR))), "MATERIALS_DIR folder does not exists"
assert(os.path.isdir(os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2"))), "invalid folder structure"
assert(os.path.isdir(os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\documents"))), "invalid folder structure"
print("Paths validated")

### Q2 Validation

In [ ]:
assert(isinstance(INDEX_Q2, FileIndex)), "Index Type"
assert(isinstance(QP_Q2, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q2, Searcher)), "Searcher Type"
print("Q2 Types Validated")

### Q3 Validation

In [ ]:
assert(isinstance(INDEX_Q3, FileIndex)), "Index Type"
assert(isinstance(QP_Q3, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q3, Searcher)), "Searcher Type"
print("Q3 Types Validated")

### Q4 Validation (Graduate Students)

In [ ]:
assert((not GRAD_STUDENT) or isinstance(INDEX_Q4, FileIndex)), "Index Type"
assert((not GRAD_STUDENT) or isinstance(QP_Q4, QueryParser)), "Query Parser Type"
assert((not GRAD_STUDENT) or isinstance(SEARCHER_Q4, Searcher)), "Searcher Type"
print("Q4 Types Validated")